In [31]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from itertools import product
from gensim.models import word2vec
import itertools
from tqdm import tqdm 

In [24]:
# Read recipe inputs
reviews_TF_IDF = dataiku.Dataset("reviews_TF_IDF")
df = reviews_TF_IDF.get_dataframe()

In [25]:
folder_path = dataiku.Folder("m9JZdV7b").get_path()
model_path = folder_path + "/word2vec_ramen_model.model"
ramen_model = word2vec.Word2Vec.load(model_path)

In [26]:
store_cross = []
for ids in product(df['id'], repeat=2):
    store_cross.append(ids)

In [27]:
store_cross_df = pd.DataFrame(store_cross, columns=['id_x', 'id_y'])

In [28]:
store_cross_detail = store_cross_df.merge(df[['id','store_name','score','review_cnt','texts_tfidf_sorted_top20']], 
                                          how='inner', 
                                          left_on='id_x', 
                                          right_on='id').drop(columns='id').merge(df[['id','store_name','score','review_cnt','texts_tfidf_sorted_top20']], 
                                                                                  how='inner', 
                                                                                  left_on='id_y', 
                                                                                  right_on='id').drop(columns='id')

In [29]:
store_cross_detail.head()

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
0,ID-000001,ID-000001,手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
1,ID-000002,ID-000001,Homemade Ramen 麦苗,3.97,731,"['記帳', 'ボード', 'ウェイティング', '深谷', '9時', '記名', 'ワン...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
2,ID-000003,ID-000001,麺尊 RAGE,3.96,723,"['金華', 'ハム', 'ロック', '穂先', '燻製', 'レモン', '豚バラ', ...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
3,ID-000004,ID-000001,宍道湖しじみ中華蕎麦 琥珀,3.95,296,"['シジミ', 'バラ', '焼き', '紫玉ねぎ', '吊し', '菅野', 'ワンタン'...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
4,ID-000005,ID-000001,らぁ麺や 嶋,3.95,177,"['記帳', '4点', '細切り', '記入', '炭火焼き', '昆布', '燻製', ...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."


In [30]:
# store_cross_detail = store_cross_detail[store_cross_detail['id_x'].isin(df['id'].loc[0:50])]
store_cross_detail = store_cross_detail.reset_index(drop=True).sort_values(['id_x'])

In [32]:
##ラーメン店xに対してラーメン店yの類似度を算出
#コサイン類似度を算出する関数を定義
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [39]:
#cossimだけの組み合わせ（同じワード同士の組みあわせがでてくるため）
#2次元を１次元にする　setが重複を削除てきなやつ。
uniq_words = list(set(itertools.chain.from_iterable(df['texts_tfidf_sorted_top20'].values)))
scores = {}
for word1, word2 in product(uniq_words, repeat=2):
    print(word1, word2)
    scores[(word1, word2)] =  cos_sim(ramen_model.wv[word1], ramen_model.wv[word2])

素 素


KeyError: "word '素' not in vocabulary"

In [50]:
df['texts_tfidf_sorted_top20'].values.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [16]:
avg_avg_scores = []
for i in tqdm(range(len(store_cross_detail['texts_tfidf_sorted_top20_x']))):
    avg_scores = []
    for j in range(len(store_cross_detail['texts_tfidf_sorted_top20_x'][i])):
        word_cross_scores = []
        word_a = store_cross_detail['texts_tfidf_sorted_top20_x'][i][j]
        for k in range(len(store_cross_detail['texts_tfidf_sorted_top20_y'][i])):
            word_b = store_cross_detail['texts_tfidf_sorted_top20_y'][i][k]
            score = scores[(word_a, word_b)]#単語間のスコアを出す。
            word_cross_scores.append(score)
        avg_scores.append(np.mean(word_cross_scores))#20個の単語間スコアの平均値
    avg_avg_scores.append(np.mean(avg_scores))#20個の単語間スコアの平均値の平均値
store_cross_detail.insert(6, 'avg_cos_sim_rate', avg_avg_scores)  
# 「二郎」と類似度が高いラーメン屋を高い順に表示
store_cross_detail = store_cross_detail.sort_values(['id_x', 'avg_cos_sim_rate'], ascending=[True, False])
df_sim_x = store_cross_detail[store_cross_detail['store_name_x'].str.contains('二郎')]
df_sim_x.reset_index(drop=True)

def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result
b = df_sim_x['avg_cos_sim_rate']
c = min_max(b.values)
df_sim_x.insert(7, '正規化', c)
df_sim_x

NameError: name 'store_df' is not defined

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

similarity_scores_df = reviews_TF_IDF_df # For this sample code, simply copy input to output


# Write recipe outputs
similarity_scores = dataiku.Dataset("similarity_scores")
similarity_scores.write_with_schema(similarity_scores_df)